# Kopier  punkt-geometri fra systemobjekt til morobjekt + datter-hierarki

Ting vi må ta hensyn til: 
  * Vegobjekt med eksisterende egengeometri (uansett hvilken type geometri) skal ikke røres
    * Vi traverserer relasjonstreet og dropper de casene der ett objekt i relasjonstreet har egengeometri
  * Vi må endre alle datterobjekter, slik at de får samme punkt-geometri som rot-objektet i relasjonen
  * Vi må håndtere at flere av våre kandidater inngår i samme relasjonstre, og må passe oss for at samme objekt ikke blir sendt inn to ganger (en gang fordi 97 variabelt skilt  [540008149](https://nvdbapiles-v3.atlas.vegvesen.no/vegobjekter/97/540008149/1) inngår i relasjonstreet til skiltpunktet [540008156](https://nvdbapiles-v3.atlas.vegvesen.no/vegobjekt.json?id=540008156&dybde=full), og én gang fordi variabelt skilt 540008149 har en selvstendig oppføring i datasettet
    * => Valgt løsning: Ignorer alle vegobjekt som ikke er øverst i relasjonstreet. 
    
    
    
## Testcase: 

Filtrert på vegsystemrefeanse [FV156 S1D1 m1000-1200](https://vegkart.atlas.vegvesen.no/#kartlag:geodata/@263125,6631475,11/hva:!(id~794)~/hvor:(vegsystemreferanse~!FV156S1D1M1000-1200)~) som gir 13 vegsystem-objekter, og som også omfatter at vi har denne casen med at et variabelt skilt som inngår to steder i datasettet, fordi både mor-objektet skiltpunkt og datterobjekt Variabelt skilt har hvert sitt systemobjekt. 

Merk at vi på vegsystemreferanse [Fv156 S1D1 m1160](https://vegkart.atlas.vegvesen.no/#kartlag:topo4/@265342,6632373,18/hva:!(id~794)~/hvor:(vegsystemreferanse~!FV156S1D1M1160-1162)~) har tre systemobjekter med skiltpunkt-variabelt skilt (dvs dobbeltregistrering av både skiltpunkt og variabelt skilt: 

Det ene skiltpunktet har et underliggende variabelt skilt som IKKE har systemobjekt

  * Skiltpunkt 540008153 med egenskaper _Oppsettingsutstyr=Stolpe 60,3 mm (2"")_, _Eier=Fylkeskommune_, _Vedlikeholdsansvarlig=SVV_ og relasjonstre:
    * 97 Variabelt skilt 540008154 med egenskaper _Variasjon=Ett motiv på/av_ og _Type=Mekanisk_
      * Dette skiltet har ingen døttre, og heller ikke noe tilknyttet systemobjekt
    * 794 Systemobjekt 540008155

Det andre skiltpunktet har et variabelt skilt med systemobjekt

  * Skiltpunkt 540008156 med egenskaper _Oppsettingsutstyr=Stolpe 60,3 mm (2"")_ og relasjonstre:
    * 97 Variabelt skilt 540008149 med egenskaper _Fjernstyring=Nei_, _Type=Mekanisk med lameller_
      * 761 Tilstand/skade punkt med egenskaper _Tilstand=OK_ 
        * 762 Tilstand/skade FU punkt med egenskaper _Etterslep=Nei_ 
        * 794 Systemobjekt 540008152 med _Beskrivelse_-egenskap 
   * 794 Systemobjekt 540008157 med _Beskrivelse_-egenskap
   
Dette er opplagt en dobbeltregistrering, men det er utafor dagens tema å rydde opp i dobbeltregistreringer

### Metodikk 

  * Vi itererer over hvert enkelt systemobjekt og finner mor-objektet (med alle relasjoner)
  * Datakontroll mor-objekt + systemobjekt: 
    * Er det pussig stor avstand mellom systemobjekt og mor-objektet så tyder det på en datafeil? => FORKAST+loggfør
    * Vi ignorerer de casene der systemobjektets mor-objekt ikke er på toppen av sitt relasjonstre
      * UNNTAK: Belysningspunkt, som kan være datter til en belysningsstrekning.
      * Andre unntak fra denne regelen? Punkt som datter til linje må da være OK?  
  * Vi traverserer relasjonstreet og gjør følgende: 
    * Sjekker om vi finner egengeometri for noen av objektene i relasjonstreet => FORKAST HELE RELASJONSTREET (loggføres)
    * Sjekker at alle objekter i relasjonstreet står samme sted => HVIS IKKE: FORKAST HELE RELASJONSTREET (loggføres)
      * Sjekk via geografisk avstand? 
      * Sjekk via posisjon på veglenkesekvens?
    * Lagrer den informasjon vi trenger for å modifisere mor-objekt + alle objekter i relasjonstreet
      * Objekt type ID
      * Objekt ID
      * Versjons ID
      * Id til egenskapen "Geometri, punkt" (hentes fra datakatalogen) 

# Skal vi ta alle objekttyper, eller dropper vi noen? 

Dette er de objekttypene der "Geometri, punkt" tilfører mest verdi. 


### Legger "Geometri, punkt" på disse

  * 95 Skiltpunkt: Geometri, punkt                44260 stk
  * 83 Kum: Geometri, punkt                       24948 stk
  * 87 Belysningspunkt: Geometri, punkt           13316 stk
  * 98 Referansestolpe: Geometri, punkt            4882 stk
  * 27 Renovasjon: Geometri, punkt                  574 stk
  * 342 Trafikkspeil: Geometri, punkt               129 stk
  * 28 Utemøbler: Geometri, punkt                   116 stk
  * 90 Signalpunkt: Geometri, punkt                 111 stk
  * 181 Lysmast: Geometri, punkt                     99 stk
  * 502 Skap, teknisk: Geometri, punkt               82 stk
  * 623 Stativ for turistinfo: Geometri, punkt       63 stk
  * 163 Kamera: Geometri, punkt                      23 stk
  * 542 Støtpute: Geometri, punkt                    14 stk
  * 165 Vindmåler: Geometri, punkt                    5 stk
  * 210 Pumpestasjon: Geometri, punkt                 4 stk
  * 481 Tennpunkt: Geometri, punkt                    4 stk
  * 85 Pumpe: Geometri, punkt                         4 stk
  * 461 Elektrisk anlegg: Geometri, punkt             2 stk
  * 97 Variabelt skilt: Geometri, punkt               1 stk

### Stikkrenner, hva gjør vi med dem?

Stikkrenner er ekstremt nyttige å ha geometri på. Formodentlig er punktet representativt for enten innløp eller utløp - gir dette verdi? Skal vi anvende metodikken på de 39624 objektene? Merk at vi kun føyer til punkt-geometri kun dersom det ikke finnes noen egengeometri fra før (punkt, linje eller flate)

  * 79 Stikkrenne/Kulvert: Geometri, punkt og Geometri, linje og Geometri, flate    39624 stk

### Diskuterbar: 

Punkt-geometri kan tilføre verdi såfremt vi ikke har annen geometri (linje eller flate) 

  * 25 Leskur: Geometri, punkt og Geometri, flate og Utgår_Geometri, linje            632 stk
  * 65 Bygning: Geometri, punkt og Utgår_Geometri, linje og Geometri, flate            21 stk
  * 73 Sikringsbolt: Geometri, punkt og Geometri, linje og Geometri, flate              7 stk

### Foreslår disse droppes

Dette er objekter der nytteverdien ikke er så stor ved å føye til "Geometri, punkt"

  * 103 Fartsdemper: Geometri, punkt                                                  419 stk
  * 23 Vegbom: Geometri, punkt og Geometri, linje                                     166 stk


### Disse har linjegeometri og punktet tilfører liten verdi

  * 7 Gjerde: Geometri, linje                                                        1349 stk
  * 20 Kantstolper/Refleks: Geometri, linje                                           870 stk
  * 174 Gangfelt: Geometri, punkt og Geometri, linje og Geometri, flate               742 stk
  * 3 Skjerm: Geometri, linje                                                         579 stk
  * 519 Vegoppmerking, tverrgående: Geometri, punkt og Geometri, linje                107 stk
  * 22 Ferist: Geometri, punkt og Geometri, linje                                      62 stk
  * 24 Skiltportal: Geometri, punkt og Geometri, linje                                 22 stk

### Og dette her er jo utdatert: 

  * 465 Utgår_Matepunkt: Utgår_Geometri, punkt                                         81 stk

In [137]:
import json
import requests
import re
import warnings
warnings.simplefilter('ignore', UserWarning )
from datetime import datetime


import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import wkt
from shapely.geometry.point import Point
import xmltodict
from xml.parsers.expat import ExpatError

import STARTHER
import nvdbapiv3
import skrivnvdb


In [2]:
sok = nvdbapiv3.nvdbFagdata( 794)
sok.filter( {'vegsystemreferanse' : 'FV156 S1D1 m1000-1200' })
mydf = pd.DataFrame( sok.to_records( relasjoner=True ))

In [3]:
def hvilkeForeldreTyper( rel:dict): 
    returnval = '-99 Uten foreldre'
    
    if not pd.isnull( rel ) and 'foreldre' in rel: 
        temp = [ str( x['type']['id'] ) + ' ' + str( x['type']['navn'] )  for x in rel['foreldre'] ]
        temp.sort()
        returnval = ','.join( temp )
        
    return returnval 

def hvilkenForeldreId( rel:dict ): 
    returnval = ''

    if not pd.isnull( rel ) and 'foreldre' in rel: 
        if len( rel['foreldre'] ) == 1: 
            if len( rel['foreldre'][0]['vegobjekter']) == 1: 
                returnval = str( rel['foreldre'][0]['vegobjekter'][0] )
            else: 
                print(f"Fant {len(rel['foreldre'][0]['vegobjekter'])} mor-objekter" )
                tmp = [str(x) for x in rel['foreldre'][0]['vegobjekter'] ]
                returnval = ','.join( tmp )
        else: 
            print( f"Fant {rel['foreldre']} ulike typer mor-relasjon" )
                
    return returnval 

# Dekoder med regulære uttrykk

In [17]:
def dekodbeskrivelse2( beskr:str, returtype='wkt' ):
    
    nord    = grabtekst( beskr, 'nord'  )
    oest    = grabtekst( beskr, 'oest' )
    hoyde   = grabtekst( beskr, 'hoyde' )
    mode    = grabtekst( beskr, 'mode' )    
    cq      = grabtekst( beskr, 'cq' ) 
    avstand = grabtekst( beskr, 'avstand' )
    
    if isinstance( nord, str) and isinstance( oest, str): 
        if isinstance( hoyde, str): 
            wkt = f"POINT Z( {oest} {nord} {hoyde} )" 
        else: 
            wkt = f"POINT Z( {oest} {nord} )" 
    elif isinstance( nord, list) and isinstance( oest, list):

        if isinstance( hoyde, list ): 
            koordstr = 'LINESTRING Z( '
            for ix, junk in enumerate( oest):
                if len( nord ) >= ix+1 and len( hoyde ) >= ix+1: 
                    koordstr += f" {oest[ix]} {nord[ix]} {hoyde[ix]}"
                    
                    if len( oest ) > ix+1: 
                        koordstr += ' , '
                        
        else: 
            koordstr = 'LINESTRING ( '
            for ix, junk in enumerate( oest):
                if len( nord ) >= ix+1 : 
                    koordstr += f" {oest[ix]} {nord[ix]}"
                    
                    if len( oest ) > ix+1: 
                        koordstr += ' , '
            
        if len( koordstr ) > len( 'LINESTRING Z(   ' ):
            koordstr += ' )'
            wkt = koordstr 
        else: 
            wkt = 'LINESTRING EMPTY'
                
    else: 
        wkt = 'POINT EMPTY'
        
    return { 'nord' : nord, 'oest' : oest, 'hoyde' : hoyde, 'mode' : mode, 'cq' : cq, 'avstand' : avstand, 'wkt' : wkt  }
    

def grabtekst( beskr:str, tagg:str ): 
    
    tmp = re.findall( f"<{tagg}>(.*?)</{tagg}>", beskr  )
    if len( tmp ) == 1: 
        return tmp[0]
    elif len( tmp ) > 1: 
        # print( f"Fant flere enn enn forekomst av {tagg} i {beskr}")
        return tmp
    else: 
        return None 

In [22]:
mydf['Beskrivelse'].fillna( '<vegreg_frapos/>', inplace=True)

mydf['dekodet'] = mydf['Beskrivelse'].apply( dekodbeskrivelse2 )

mydf['dekodet_geom'] = mydf['dekodet'].apply( lambda x : x['wkt'] )

# Kvalitetssikring og filtrering

In [25]:
godkjente = mydf[ mydf['dekodet_geom'].str.contains( 'POINT')].copy()

godkjente = godkjente[ ~godkjente['morObjektId'].str.contains(',')] # Ignorerer dem med flere enn ett morobjekt

godkjente = godkjente[ godkjente['morObjektId'] != ''] # Ignorerer dem uten morobjekt
godkjente = godkjente[ ~godkjente['morObjektId'].isnull()].copy() 

godkjente['morObjektId'] = godkjente['morObjektId'].astype(int)

# Finner foreldrerelasjon til systemobjektet

In [26]:
godkjente['morObjektId'] = godkjente['relasjoner'].apply( hvilkenForeldreId )

In [27]:
godkjente['hvilkeForeldre'] = godkjente['relasjoner'].apply( hvilkeForeldreTyper )

# Itererer over alle systemobjekt

In [104]:
def traverserRelasjoner( relasjonstre:dict, kontrollgeom:Point  ): 
    """
    Finner NVDB ID, versjons ID og egenskapstypeId for punktgeometri for alle medlemmer i relasjonstre
    
    Vil traversere "barn" - elementet i relasjonstreet rekursivt 
    
    ARGUMENTS: 
        relasjonstre - dictionary, "barn" - elementet i et vegobjekts relasjonstre (fra NVDB api LEs)
        
        kontrollgeom - Shapely Point - objekt 
        
        dakat - JSON-representasjon av datakatalogen 
    
    RETURNS:
        (suksess, vegobjekter) 
        
            suksess: set (mengde) med feilmeldinger fra testene. En tom mengde betyr at alt er OK
            endringsliste: Liste med tupler (vegobjektTypeId, vegobjektId, versjonsId)
    """
    
    endringsliste = []
    suksess = set()
    
    
    for enRelasjonstype in relasjonstre: 
        for etObjekt in enRelasjonstype['vegobjekter']: 
            
            
            nyttObj = nvdbapiv3.nvdbFagObjekt( etObjekt )
            egengeom = nyttObj.egenskap( 'eometri')
            if egengeom: 
                suksess.add( f"Fant egengeometri for objekttype {mittobj['metadata']['type']['id']} objekttype {mittobj['metadata']['type']['navn']}")
            
            # Flere tester nedover!
            mittObjGeom = wkt.loads( etObjekt['geometri']['wkt'] )
            avst =  kontrollgeom.distance(mittObjGeom) 
            if avst > 0.5: 
                if avst <= 1: 
                    avstand = '0-1m'
                elif avst <= 5: 
                    avstand = '1-5m'
                elif avst <= 10: 
                    avstand = '5-10m'
                elif avst <= 50:
                    avstand = '10-50m'
                else: 
                    avstand = '>50m'
                suksess.add( f'Senterlinjegeometri avviker {avstand}')
                

            if etObjekt['metadata']['type']['id'] not in  [794]: # 794 Systemobjekt, dette ignorerer vi
                endringsliste.append( ( etObjekt['metadata']['type']['id'], etObjekt['id'], etObjekt['metadata']['versjon'] )  )  
             
            # Rekursiv 
            if 'relasjoner' in etObjekt and 'barn' in etObjekt['relasjoner']: 
                
                (minSuksess, minEndringsliste) = traverserRelasjoner( etObjekt['relasjoner']['barn'] , kontrollgeom )
                suksess.update( minSuksess )
                endringsliste.extend( minEndringsliste )
    
    return (suksess, endringsliste)  


    

In [129]:
forb = nvdbapiv3.apiforbindelse()

har_foreldre = []
geometriavvik = []
har_egengeometri = []
endringsmanus = []
antall_vegobjekt = 0

for ix, row in godkjente.iterrows(): 
    aksepter = True 
    
    mittobj = forb.les( '/vegobjekt', params={'id' : row['morObjektId'], 'dybde' : 'full'}  )
    if mittobj.ok: 
        mittobj = mittobj.json()
 
        # Sjekker at vi er på rot-nivå i relasjonstreet, eller at vi er på en objekttype der det er greit 
        if 'foreldre' in mittobj['relasjoner']: 
            OK_foreldre = [ 86   ] # 86 Belysningsstrekning
            tmp = { x['type']['id'] : x['type']['navn']  for x in   mittobj['relasjoner']['foreldre'] } 
            if list( tmp.keys())[0] not in OK_foreldre:             
                aksepter = False
                har_foreldre.append( row.to_dict())

        # Geometrikontroll
        geomsted   = wkt.loads( row['geometri'])
        geomNyEgen = wkt.loads( row['dekodet_geom'])

        # Avstand mellom ny egengeometri og stedfestingsgeometri
        if geomsted.distance( geomNyEgen ) > 10: 
            aksepter = False
            geometriavvik.append( row.to_dict())

        # Har vi geometri?
        fancyObj = nvdbapiv3.nvdbFagObjekt( mittobj )
        if fancyObj.egenskap( 'eometri'):
            aksepter = False 
            har_egengeometri.append( mittobj )


        if aksepter:
            dette_treet = [ ( mittobj['metadata']['type']['id'], mittobj['id'], mittobj['metadata']['versjon']  ) ]
            (suksess, vegobjekt) = traverserRelasjoner( mittobj['relasjoner']['barn'], geomsted)
            dette_treet.extend( vegobjekt )
            endringsmanus.append( { 'systemobjektId' : row['nvdbId'], 
                                 'hvilkeForeldre' : row['hvilkeForeldre'], 
                                 'vref'   : row['vref'], 
                                 'ny_egengeom' : row['dekodet_geom'],
                                 'vegnettsgeom' : row['geometri'],
                                 'endringssett' :  dette_treet } ) 
            antall_vegobjekt += len( dette_treet )

    else: 
        print( f"Fikk feilmelding på henting av objekt {row['morObjektId']}" )
        aksepter = False 

        
print( f"{len(endringsett)} relasjonstrær med {antall_vegobjekt} vegobjekt totalt" )
print( f"  Forkastet pga foreldre {len(har_foreldre)}") 
print( f"  Forkastet pga geometriavvik {len(geometriavvik)}")
        

10 relasjonstrær med 27 vegobjekt totalt
  Forkastet pga foreldre 1
  Forkastet pga geometriavvik 0


In [124]:
row

objekttype                                                         794
nvdbId                                                       540008158
versjon                                                              1
startdato                                                   2014-08-14
Beskrivelse          <vegreg_frapos><gnss_data><talker>GN</talker><...
relasjoner           {'foreldre': [{'listeid': 221899, 'id': 201899...
veglenkesekvensid                                               444218
detaljnivå                                       Vegtrase og kjørebane
typeVeg                                                   Enkel bilveg
kommune                                                           3020
fylke                                                               30
vref                                                  FV156 S1D1 m1167
veglenkeType                                                     HOVED
vegkategori                                                          F
fase  

In [103]:
godkjente[[ 'hvilkeForeldre', 'morObjektId']]

,hvilkeForeldre,morObjektId
0,87 Belysningspunkt,83796192
1,98 Referansestolpe,540008138
2,87 Belysningspunkt,83796196
3,87 Belysningspunkt,83796200
4,95 Skiltpunkt,84161094
5,87 Belysningspunkt,83796204
7,83 Kum,84161102
9,97 Variabelt skilt,540008149
10,95 Skiltpunkt,540008153
11,95 Skiltpunkt,540008156


In [101]:
godkjente.columns

Index(['objekttype', 'nvdbId', 'versjon', 'startdato', 'Beskrivelse',
       'relasjoner', 'veglenkesekvensid', 'detaljnivå', 'typeVeg', 'kommune',
       'fylke', 'vref', 'veglenkeType', 'vegkategori', 'fase', 'vegnummer',
       'relativPosisjon', 'adskilte_lop', 'trafikantgruppe', 'geometri',
       'morObjektId', 'hvilkeForeldre', 'dekodet', 'dekodet_geom'],
      dtype='object')

In [125]:
endringsett

[{'systemobjektId': 540008137,
  'hvilkeForeldre': '87 Belysningspunkt',
  'vref': 'FV156 S1D1 m1000',
  'ny_egengeom': 'POINT Z( 265445.287 6632470.400 )',
  'endringssett': [(87, 83796192, 3), (88, 83796190, 3), (181, 83796191, 2)]},
 {'systemobjektId': 540008141,
  'hvilkeForeldre': '98 Referansestolpe',
  'vref': 'FV156 S1D1 m1000',
  'ny_egengeom': 'POINT Z( 265445.287 6632470.400 )',
  'endringssett': [(98, 540008138, 1),
   (761, 540008139, 1),
   (762, 540008140, 1)]},
 {'systemobjektId': 540008142,
  'hvilkeForeldre': '87 Belysningspunkt',
  'vref': 'FV156 S1D1 m1041',
  'ny_egengeom': 'POINT Z( 265414.663 6632443.301 )',
  'endringssett': [(87, 83796196, 3), (88, 83796194, 2), (181, 83796195, 2)]},
 {'systemobjektId': 540008143,
  'hvilkeForeldre': '87 Belysningspunkt',
  'vref': 'FV156 S1D1 m1083',
  'ny_egengeom': 'POINT Z( 265382.987 6632415.896 )',
  'endringssett': [(87, 83796200, 3), (88, 83796198, 2), (181, 83796199, 2)]},
 {'systemobjektId': 540008144,
  'hvilkeForeld

# Henter datakatalogen, lager endringssett til SKRIV

In [133]:
dakat1 = forb.les( '/vegobjekttyper', 
                      params={'inkluder' : 'egenskapstyper'} ).json()

dakat = { }
for objType in dakat1: 
    dakat[objType['id']] = {}
    geometriegenskaper = [ x for x in objType['egenskapstyper'] if 'eometri' in x['navn']]
    for geom in geometriegenskaper: 
        dakat[objType['id']][geom['navn']] = geom['id']
            


In [134]:
alle_endringssett = []

max_antall = 20
count = 0 

etEndringsset = skrivnvdb.endringssett_mal( )

for relasjonstre in endringsmanus: 
    count += len( relasjonstre['endringssett'])
    
    for objekt in relasjonstre['endringssett']: 
        
        if 'Geometri, punkt' in dakat[objekt[0]]:
        
            etEndringsset['delvisOppdater']['vegobjekter'].append({ 
                'gyldighetsperiode' : { 'startdato' : datetime.now().isoformat()[0:10] },
                'typeId' : objekt[0],
                'nvdbId' : objekt[1],
                'egenskaper' : [ { 'typeId' : dakat[objekt[0]]['Geometri, punkt'],  }]
            })
    

In [136]:
etEndringsset

{'datakatalogversjon': '2.29', 'delvisOppdater': {'vegobjekter': []}}

In [142]:
objekt = relasjonstre['endringssett'][0]
dakat[objekt[0]]['Geometri, punkt']

4787

In [135]:
relasjonstre

{'systemobjektId': 540008158,
 'hvilkeForeldre': '87 Belysningspunkt',
 'vref': 'FV156 S1D1 m1167',
 'ny_egengeom': 'POINT Z( 265316.256 6632365.581 )',
 'vegnettsgeom': 'POINT Z(265317.582 6632363.587 84.749)',
 'endringssett': [(87, 83796208, 3), (88, 83796206, 2), (181, 83796207, 2)]}

In [ ]:
kandidater = [ 95, 79, 83, 87, 98, 7, 174, 25, 27, 103, 20, 23, 3, 342, 28, 181, 90, 519, 623, 22, 163, 24, 65, 542, 73, 85, 210, 465, 97, 165, 502 ]
med_punkt = []
uten_punkt = []

for kandidat in kandidater: 
    if 'Geometri, punkt' in dakat[kandidat]:
        med
    
    

In [143]:
dakat[95]

{'Geometri, punkt': 4794}